In [2]:
import re
import os 
import json

In [3]:
file_name="cctv_camera.json"
base_name=os.path.splitext(os.path.basename(file_name))[0] #This takes only the file name part not the extension part

In [4]:
with open(file_name,'r',encoding="utf-8") as f:
    data = json.load(f)

In [5]:
item_list = data["mods"]["listItems"]

In [6]:
def chunking_each_doc(doc,chunk_size = 128):
    start = 0
    doc = json.dumps(doc)
    tokens = re.findall(r'\w+|[{}[\]:,",]',doc)
    chunks = []
    while start <= len(tokens):
        end = min(start+chunk_size,len(tokens))
        chunk = " ".join(tokens[start:end])
        chunks.append(chunk)
        start+=chunk_size
    return chunks

In [7]:
all_chunks = []
for item in item_list:
    chunks_of_each_doc= chunking_each_doc(item)
    all_chunks.append(chunks_of_each_doc)

In [8]:
len(all_chunks)

40

In [9]:
all_chunks[0][2]

'" , " current_price " : " 4359 " , " x_object_id " : " 366335641 " } , " originalPriceShow " : " " , " priceShow " : " Rs 4 , 359 " , " ratingScore " : " 5 0 " , " review " : " 7 " , " location " : " Bagmati Province " , " description " : [ " Storage Options : Supports memory cards up to 128GB and cloud storage " , " Advanced Night Vision : 6 Dual Core LEDs for the right and left cameras , 11 Dual Core LEDs for the PTZ camera Includes 3 modes : Full Color , IR , and Intelligent Night Vision with auto detection and remote controlled white LED'

In [10]:
all_chunks[1]

['{ " name " : " EZVIZ H6C CS H6C 2Mp Smart Panaromic View Camera " , " nid " : " 128843472 " , " itemId " : " 128843472 " , " icons " : [ { " domClass " : " 150565 " , " type " : " img " , " group " : " 6 " , " showType " : " 0 " } , { " domClass " : " 116088 " , " type " : " img " , " group " : " 2 " , " showType " : " 0 " } , { " domClass " : " 150318 " , " text " : " Gems save Rs 92 " , " type',
 '" : " text " , " group " : " 2 " , " showType " : " 0 " } , { " domClass " : " 175175 " , " type " : " img " , " group " : " 3 " , " showType " : " 0 " } ] , " image " : " https : static 01 daraz com np p 0be43cf2e6fb25f1770e227ce52cb2a2 jpg " , " isSmartImage " : false , " utLogMap " : { " srp_name " : " LazadaMainSrp " , " x_object_type " : " item " , " src " : " organic " , " trafficType " : " organic " , " x_sku_ids " : " 1036762079 " , " x_item_ids',
 '" : " 128843472 " , " iconList " : " 150565 116088 150318 175175 " , " SN " : " f2c4c23a4ec97433ccd27c0620ae54e5 " , " current_price 

In [11]:
from qdrant_client import QdrantClient,models
from sentence_transformers import SentenceTransformer
client=QdrantClient(url="http://localhost:6333")
encoder = SentenceTransformer("all-MiniLM-L6-v2")

c:\Users\Asus\OneDrive\Desktop\AutoLab\env_name\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
dense_vector_name="dense"
sparse_vector_name="sparse"
dense_model_name="sentence-transformers/all-MiniLM-L6-v2"
sparse_model_name="prithivida/Splade_PP_en_v1"

In [23]:
if not client.collection_exists("hybrid_search_daraz_items"):
    client.create_collection(
        collection_name = "hybrid_search_daraz_items",
        vectors_config = {
            "dense" : models.VectorParams(size = client.get_embedding_size(dense_model_name), distance = models.Distance.COSINE)

        },
        sparse_vectors_config= {
            "sparse" : models.SparseVectorParams()
        }
    )

In [24]:
# creation of embeddings

#dense vectors and sparse vectors of the same document and chunks
final_dense_vector = []
final_sparse_vector = []
for i in range(len(all_chunks)):
        dense_embeddings = encoder.encode(all_chunks[i],show_progress_bar=True,convert_to_numpy=True)
        final_dense_vector.append(dense_embeddings)

Batches: 100%|██████████| 1/1 [00:00<00:00,  3.97it/s]


In [25]:
client.create_payload_index(
    collection_name="hybrid_search_daraz_items",
    field_name="doc_id",
    field_schema="integer"

)

UpdateResult(operation_id=567, status=<UpdateStatus.COMPLETED: 'completed'>)

In [26]:
client.create_payload_index(
    collection_name="hybrid_search_daraz_items",
    field_name="chunk_id",
    field_schema="integer"
)

UpdateResult(operation_id=569, status=<UpdateStatus.COMPLETED: 'completed'>)

In [27]:
client.create_payload_index(
    collection_name="hybrid_search_daraz_items",
    field_name="file_name",
    field_schema="keyword"
)

UpdateResult(operation_id=571, status=<UpdateStatus.COMPLETED: 'completed'>)

In [29]:
offset = 0
info = client.get_collection(collection_name = "hybrid_search_daraz_items")
file_number = 0
count = info.points_count
if(count!=0):
    res, _ = client.scroll(
        collection_name = "hybrid_search_daraz_items",
        with_payload = True,
        with_vectors=False,
        limit = 1,
        order_by = {
            "key":"chunk_id",
            "direction" : "desc"
        }
)
    
    if(res):
        file_number = res[0].payload.get("doc_id")[0]
        file_number = file_number + 1
        last_id = res[0].id
        offset = last_id + 1
    else:
        offset = 0
else:
    offset = 0
    file_number = 0


for doc in range(len(all_chunks)):
    for idx in range(len(all_chunks[doc])):
        client.upsert(
            collection_name = "hybrid_search_daraz_items",
            points = [
                models.PointStruct(
                    id = offset,
                    payload = {
                        "doc_id" : (file_number,doc),
                        "chunk_id":offset,
                        "chunk" : all_chunks[doc][idx],
                        "file_name" : base_name
                    },
                    vector = {
                        "dense": final_dense_vector[doc][idx].tolist(),
                        "sparse" : models.Document(text = all_chunks[doc][idx],model = sparse_model_name)
                    }
                )
            ]
        )
        offset+=1

In [21]:
class HybridSearch:
    
    def __init__(self,collection_name):
        self.collection_name = collection_name
        self.client = QdrantClient(url = "http://localhost:6333")
        self.encoder_dense = SentenceTransformer("all-MiniLM-L6-v2")
        self.sparse_model = "prithivida/Splade_PP_en_v1"
    def search_points(self,item:str):
        search_result = self.client.query_points(
            collection_name = self.collection_name,
            query = models.FusionQuery(
                fusion = models.Fusion.RRF
            ),
            prefetch=[
                models.Prefetch(
                    query= self.encoder_dense.encode([item])[0].tolist(),
                    using = "dense",
                    
                ),
                models.Prefetch(
                    query = models.Document(text = item,model = self.sparse_model),
                    using = "sparse",
                )
                ],
                query_filter=None,
                limit = 3
        ).points

        dense_result = self.client.query_points(
        collection_name=self.collection_name,
        query=self.encoder_dense.encode([item])[0].tolist(),
        using="dense",
        limit=3
        ).points

        sparse_result = self.client.query_points(
        collection_name=self.collection_name,
        query=models.Document(text=item, model=self.sparse_model),
        using="sparse",
        limit=3
        ).points

        for point in search_result:
            print("score = ",point.score, "Point_id=", point.id, "doc_id = ", point.payload.get("doc_id"), "file_name = ",point.payload.get("file_name"), "chunk_id = " , point.payload.get("chunk_id"))

In [22]:
search_item = HybridSearch("hybrid_search_daraz_items")
search_item.search_points("Rechargeable camera battery")

score =  0.64285713 Point_id= 322 doc_id =  [1, 6] file_name =  cctv_camera chunk_id =  322
score =  0.5 Point_id= 321 doc_id =  [1, 6] file_name =  cctv_camera chunk_id =  321
score =  0.5 Point_id= 45 doc_id =  [0, 6] file_name =  cctv_camera chunk_id =  45
